### Metaprogramming - Application 3

In [1]:
with open('prod.ini', 'w') as prod, open('dev.ini', 'w') as dev:
    prod.write('[Database]\n')
    prod.write('db_host=prod.mynetwork.com\n')
    prod.write('db_name=my_database\n')
    prod.write('\n[Server]\n')
    prod.write('port=8080\n')

    dev.write('[Database]\n')
    dev.write('db_host=dev.mynetwork.com\n')
    dev.write('db_name=my_database\n')
    dev.write('\n[Server]\n')
    dev.write('port=3000\n')

In [7]:
import configparser

In [8]:
class Config:
    def __init__(self, env='dev'):
        print(f'Loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        self.db_host = config['Database']['db_host']
        self.db_name = config['Database']['db_name']
        self.port = config['Server']['port']

In [9]:
config = Config('dev')

Loading config from dev file


In [11]:
config.__dict__

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database', 'port': '3000'}

In [19]:
help(config)

Help on Config in module __main__ object:

class Config(builtins.object)
 |  Config(env='dev')
 |
 |  Methods defined here:
 |
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [20]:
class Config:
    def __init__(self, env='dev'):
        print(f'Loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            for key, value in config[section_name].items():
                setattr(self, key, value)

In [21]:
config = Config('prod')

Loading config from prod file


In [22]:
vars(config)

{'db_host': 'prod.mynetwork.com', 'db_name': 'my_database', 'port': '8080'}

In [23]:
help(config)

Help on Config in module __main__ object:

class Config(builtins.object)
 |  Config(env='dev')
 |
 |  Methods defined here:
 |
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [32]:
class Section:
    def __init__(self, name, items_dict):
        """
        name: str
            name of section
        item_dict: dict
            dictionary of named (key) config values (value)
        """
        self.name = name
        for key, value in items_dict.items():
            setattr(self, key, value)

In [33]:
class Config:
    def __init__(self, env='dev'):
        print(f'Loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            section = Section(section_name, config[section_name])
            setattr(self, section_name.casefold(), section)

In [34]:
config = Config()

Loading config from dev file


In [35]:
vars(config)

{'database': <__main__.Section at 0x7fab6e4c2d50>,
 'server': <__main__.Section at 0x7fab6e4c3ef0>}

In [36]:
help(config)

Help on Config in module __main__ object:

class Config(builtins.object)
 |  Config(env='dev')
 |
 |  Methods defined here:
 |
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [37]:
class SectionType(type):
    def __new__(cls, name, bases, cls_dict, section_name, items_dict):
        cls_dict['__doc__'] = f'Configs for {section_name} section'
        cls_dict['section_name'] = section_name
        for key, value in items_dict.items():
            cls_dict[key] = value
        return super().__new__(cls, name, bases, cls_dict)

In [40]:
class DatabaseSection(metaclass=SectionType,
                      section_name='database',
                      items_dict={'db_host': 'db_host', 'db_name': 'db_name'}):
    pass

In [41]:
vars(DatabaseSection)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Configs for database section',
              'section_name': 'database',
              'db_host': 'db_host',
              'db_name': 'db_name',
              '__dict__': <attribute '__dict__' of 'DatabaseSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DatabaseSection' objects>})

In [42]:
help(DatabaseSection)

Help on class DatabaseSection in module __main__:

class DatabaseSection(builtins.object)
 |  Configs for database section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  db_host = 'db_host'
 |
 |  db_name = 'db_name'
 |
 |  section_name = 'database'



In [43]:
MyClass = type('MyClass', (object,), {})

In [44]:
MyClass

__main__.MyClass

In [45]:
type(MyClass)

type

In [46]:
MySection = SectionType('DBSection', (object,), {},
                        section_name='database',
                        items_dict={'db_host': 'db_host', 'db_name': 'db_name'})

In [47]:
MySection

__main__.DBSection

In [48]:
vars(MySection)

mappingproxy({'__doc__': 'Configs for database section',
              'section_name': 'database',
              'db_host': 'db_host',
              'db_name': 'db_name',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'DBSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DBSection' objects>})

In [49]:
help(MySection)

Help on class DBSection in module __main__:

class DBSection(builtins.object)
 |  Configs for database section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  db_host = 'db_host'
 |
 |  db_name = 'db_name'
 |
 |  section_name = 'database'



In [52]:
class ConfigType(type):
    def __new__(cls, name, bases, cls_dict, env):
        """
        env: str
            The environment we are loading for the config (e.b. prod, dev)
        """
        cls_dict['__doc__'] = f'Config for {env}'
        cls_dict['env'] = env
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            class_name = section_name.capitalize()
            class_attribute_name = section_name.casefold()
            section_items = config[section_name]
            bases = (object, )
            section_cls_dict = {}
            Section = SectionType(
                class_name, bases, section_cls_dict,
                section_name=section_name,
                items_dict=section_items
            )
            cls_dict[class_attribute_name] = Section
        return super().__new__(cls, name, bases, cls_dict)

In [53]:
class DevConfig(metaclass=ConfigType, env='dev'):
    pass

class ProdConfig(metaclass=ConfigType, env='prod'):
    pass

In [55]:
DevConfig.database.db_name

'my_database'

In [56]:
ProdConfig.server.port

'8080'

In [57]:
help(DevConfig)

Help on class DevConfig in module __main__:

class DevConfig(builtins.object)
 |  Config for dev
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  database = <class '__main__.Database'>
 |      Configs for Database section
 |
 |
 |  env = 'dev'
 |
 |  server = <class '__main__.Server'>
 |      Configs for Server section



In [58]:
help(DevConfig.database)

Help on class Database in module __main__:

class Database(builtins.object)
 |  Configs for Database section
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  db_host = 'dev.mynetwork.com'
 |
 |  db_name = 'my_database'
 |
 |  section_name = 'Database'

